## Peak analysis

Get the following features in one table:
    
1. peak daily cases (`daily_cases` and `peak_cases_date`), 
2. peak daily deaths (`daily_deaths` and `peak_deaths_date`), 
3. peakk case incidence (`daily_inc`), 
4. peak death incidence (`daily_mor`).

All applying 7-day moving average.

In [200]:
import pandas as pd
import numpy as np
from numpy import cov
from scipy.stats import pearsonr
import datetime
t = datetime.datetime.now()
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
folder = r'C:\Users\Ensheng\Desktop\Coronavirus\COVID_MMR\data\\'
output_folder =r'C:\Users\Ensheng\Desktop\Coronavirus\COVID_MMR\output\\'

In [201]:
# pre-config
cases_or_deaths = 'cases' # 'cases' 'deaths' 
ex_list = ['Alaska','American Samoa','Guam','Northern Mariana Islands',
           'Puerto Rico','Virgin Islands','Diamond Princess','Grand Princess']

### MMR data

In [202]:
# data: MMR at the county level
in_table = folder + r'US_MMR.xlsx'
mmr_df = pd.read_excel(in_table,sheet_name='county')
print(len(mmr_df))

# remove state with unified MMR
state_list = list(set(mmr_df['State']))
# detect states with the same MMR rate for all counties
one_vr_list = []
for state in state_list:
    if (len(set(mmr_df[mmr_df['State']==state]['MMR']))) == 1:
        print(state)
        one_vr_list.append(state)
mmr_df =  mmr_df[~mmr_df['State'].isin(one_vr_list)]
mmr_df = mmr_df.drop(columns=['logMMR'])

print(len(mmr_df))
mmr_df.head(3)

3101
DE
AR
MS
OH
HI
NH
NV
WV
NE
2673


,FIPS,County,State,MMR,Population
0,1001,Autauga,AL,0.9536,55504
1,1003,Baldwin,AL,0.9700,212628
2,1005,Barbour,AL,0.9283,25270


### Cases

In [203]:
ex_list = ['Alaska','American Samoa','Guam','Northern Mariana Islands',
           'Puerto Rico','Virgin Islands','Diamond Princess','Grand Princess']

# data: cases at the county level
in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
cases_df = pd.read_csv(in_table)
cases_df = cases_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key'])

# clean the table
cases_df = cases_df[~cases_df['Province_State'].isin(ex_list)]
cases_df = cases_df[~cases_df['Admin2'].isin(['Unassigned'])]
cases_df = cases_df[~cases_df['Admin2'].str.startswith('Out of')]
cases_df = cases_df[cases_df['FIPS'].notnull()]
cases_df.loc[cases_df['FIPS'].notnull(), 'FIPS'] = cases_df[cases_df['FIPS'].notnull()]['FIPS'].astype(int) # FIPS code from float to int

print("Cases! Number of counties with FIPS: " + str(len(cases_df)))
cases_df.tail(3)

Cases! Number of counties with FIPS: 3113


,FIPS,Admin2,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20
3336,56041,Uinta,Wyoming,0,0,0,0,0,0,0,...,276,276,277,278,283,283,283,283,283,283
3338,56043,Washakie,Wyoming,0,0,0,0,0,0,0,...,97,97,100,102,104,106,106,107,108,108
3339,56045,Weston,Wyoming,0,0,0,0,0,0,0,...,7,7,8,8,12,0,11,11,11,11


### Peak daily cases

In [204]:
# reframe the case dataframe
daily_df = cases_df.drop(columns=['Admin2','Province_State'])
daily_df = daily_df.set_index('FIPS')
daily_df = daily_df.stack().reset_index()
daily_df = daily_df.rename(columns={"level_1": "date",0: "cases"})
print(len(daily_df))
daily_df.head()

672408


,FIPS,date,cases
0,1001,1/22/20,0
1,1001,1/23/20,0
2,1001,1/24/20,0
3,1001,1/25/20,0
4,1001,1/26/20,0


In [205]:
# calculate the daily cases
# ref: https://stackoverflow.com/questions/48347497/pandas-groupby-diff
daily_df['daily_cases'] = daily_df.groupby(['FIPS'])['cases'].diff().fillna(0)
daily_df = daily_df.rename(columns={"date": "peak_cases_date"})
daily_df.tail()

,FIPS,peak_cases_date,cases,daily_cases
672403,56045,8/20/20,0,-12.0
672404,56045,8/21/20,11,11.0
672405,56045,8/22/20,11,0.0
672406,56045,8/23/20,11,0.0
672407,56045,8/24/20,11,0.0


In [206]:
# calculate the moving average
# ref: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
daily_df['daily_cases_7moving'] = daily_df.groupby('FIPS')['daily_cases'].transform(lambda x: x.rolling(7, 1).mean())
daily_df.tail()

,FIPS,peak_cases_date,cases,daily_cases,daily_cases_7moving
672403,56045,8/20/20,0,-12.0,-0.714286
672404,56045,8/21/20,11,11.0,0.857143
672405,56045,8/22/20,11,0.0,0.571429
672406,56045,8/23/20,11,0.0,0.571429
672407,56045,8/24/20,11,0.0,0.428571


In [207]:
# find the peak
daily_df = daily_df.sort_values(["FIPS", "daily_cases_7moving"], ascending = (True,False))
daily_df = daily_df.groupby('FIPS').first().reset_index()
daily_df.head()

,FIPS,peak_cases_date,cases,daily_cases,daily_cases_7moving
0,1001,8/9/20,1169,83.0,22.714286
1,1003,7/23/20,2423,259.0,120.571429
2,1005,7/16/20,444,18.0,11.000000
3,1007,8/9/20,438,12.0,10.571429
4,1009,7/25/20,569,17.0,21.142857


In [208]:
df1 = daily_df
print(len(df1))
df1.head()

3113


,FIPS,peak_cases_date,cases,daily_cases,daily_cases_7moving
0,1001,8/9/20,1169,83.0,22.714286
1,1003,7/23/20,2423,259.0,120.571429
2,1005,7/16/20,444,18.0,11.000000
3,1007,8/9/20,438,12.0,10.571429
4,1009,7/25/20,569,17.0,21.142857


### Daily deaths

In [209]:
# data: deaths at the county level
in_table = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
deaths_df = pd.read_csv(in_table)
deaths_df = deaths_df.drop(columns=['UID', 'iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key','Population'])

# clean the table
deaths_df = deaths_df[~deaths_df['Province_State'].isin(ex_list)]
deaths_df = deaths_df[~deaths_df['Admin2'].isin(['Unassigned'])]
deaths_df = deaths_df[~deaths_df['Admin2'].str.startswith('Out of')]
deaths_df = deaths_df[deaths_df['FIPS'].notnull()]
deaths_df.loc[deaths_df['FIPS'].notnull(), 'FIPS'] = deaths_df[deaths_df['FIPS'].notnull()]['FIPS'].astype(int) # FIPS code from float to int

print("Deaths! Number of counties with FIPS: " + str(len(deaths_df)))
deaths_df.tail(3)

# reframe the case dataframe
daily_df = deaths_df.drop(columns=['Admin2','Province_State'])
daily_df = daily_df.set_index('FIPS')
daily_df = daily_df.stack().reset_index()
daily_df = daily_df.rename(columns={"level_1": "date",0: "deaths"})
print(len(daily_df))
daily_df.head()

# calculate the daily deaths
# ref: https://stackoverflow.com/questions/48347497/pandas-groupby-diff
daily_df['daily_deaths'] = daily_df.groupby(['FIPS'])['deaths'].diff().fillna(0)
daily_df = daily_df.rename(columns={"date": "peak_deaths_date"})
daily_df.tail()

# calculate the moving average
# ref: https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
daily_df['daily_deaths_7moving'] = daily_df.groupby('FIPS')['daily_deaths'].transform(lambda x: x.rolling(7, 1).mean())
daily_df.tail()

# find the peak
daily_df = daily_df.sort_values(["FIPS", "daily_deaths_7moving"], ascending = (True,False))
daily_df = daily_df.groupby('FIPS').first().reset_index()
daily_df.head()

Deaths! Number of counties with FIPS: 3113
672408


,FIPS,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving
0,1001,7/15/20,18,1.0,0.857143
1,1003,8/3/20,23,1.0,0.857143
2,1005,7/20/20,4,1.0,0.285714
3,1007,8/8/20,5,1.0,0.428571
4,1009,7/29/20,3,2.0,0.285714


In [210]:
df2 = daily_df
print(len(df2))
df2.head()

3113


,FIPS,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving
0,1001,7/15/20,18,1.0,0.857143
1,1003,8/3/20,23,1.0,0.857143
2,1005,7/20/20,4,1.0,0.285714
3,1007,8/8/20,5,1.0,0.428571
4,1009,7/29/20,3,2.0,0.285714


### Merge with MMR

In [211]:
df3 = pd.merge(df1,df2, how='left', on='FIPS')
print(len(df3))
df3.head(3)

3113


,FIPS,peak_cases_date,cases,daily_cases,daily_cases_7moving,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving
0,1001,8/9/20,1169,83.0,22.714286,7/15/20,18,1.0,0.857143
1,1003,7/23/20,2423,259.0,120.571429,8/3/20,23,1.0,0.857143
2,1005,7/16/20,444,18.0,11.000000,7/20/20,4,1.0,0.285714


In [212]:
# counties with MMR but no cases
mmr_df[mmr_df['FIPS'].isin(set(mmr_df['FIPS']) - set(df3['FIPS']))]

,FIPS,County,State,MMR,Population


In [213]:
# counties without MMR
df3[df3['FIPS'].isin(set(df3['FIPS']) - set(mmr_df['FIPS']))][['FIPS']]

,FIPS
82,5001
83,5003
84,5005
85,5007
86,5009
...,...
3013,54101
3014,54103
3015,54105
3016,54107


In [214]:
df4 = pd.merge(mmr_df,df3, how='left', on='FIPS')
print(len(df4))
df4.head(3)

2673


,FIPS,County,State,MMR,Population,peak_cases_date,cases,daily_cases,daily_cases_7moving,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving
0,1001,Autauga,AL,0.9536,55504,8/9/20,1169,83.0,22.714286,7/15/20,18,1.0,0.857143
1,1003,Baldwin,AL,0.9700,212628,7/23/20,2423,259.0,120.571429,8/3/20,23,1.0,0.857143
2,1005,Barbour,AL,0.9283,25270,7/16/20,444,18.0,11.000000,7/20/20,4,1.0,0.285714


### Calculate incidence rate and mortality

In [224]:
df4['daily_inc'] = df4['daily_cases_7moving'] * 100000 / df4['Population']
df4['daily_mor'] = df4['daily_deaths_7moving'] * 100000 / df4['Population']
print(len(df4))
df4.head(3)

2673


,FIPS,County,State,MMR,Population,peak_cases_date,cases,daily_cases,daily_cases_7moving,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving,daily_inc,daily_mor
0,1001,Autauga,AL,0.9536,55504,8/9/20,1169,83.0,22.714286,7/15/20,18,1.0,0.857143,40.923691,1.544290
1,1003,Baldwin,AL,0.9700,212628,7/23/20,2423,259.0,120.571429,8/3/20,23,1.0,0.857143,56.705339,0.403119
2,1005,Barbour,AL,0.9283,25270,7/16/20,444,18.0,11.000000,7/20/20,4,1.0,0.285714,43.529877,1.130646


### Calculate correlation

In [243]:
df = df4
state_list = list(set(df['State']))
state_list.sort(reverse=False)

field_name = 'daily_deaths_7moving'
df = df[df[field_name]!=0]
df = df[df['MMR']>=0]
df.loc[:,('MMR')] = np.log(df['MMR'])
# df.loc[:,(field_name)] = np.log(df[field_name])
df

,FIPS,County,State,MMR,Population,peak_cases_date,cases,daily_cases,daily_cases_7moving,peak_deaths_date,deaths,daily_deaths,daily_deaths_7moving,daily_inc,daily_mor
0,1001,Autauga,AL,-0.047511,55504,8/9/20,1169,83.0,22.714286,7/15/20,18,1.0,0.857143,40.923691,1.544290
1,1003,Baldwin,AL,-0.030459,212628,7/23/20,2423,259.0,120.571429,8/3/20,23,1.0,0.857143,56.705339,0.403119
2,1005,Barbour,AL,-0.074400,25270,7/16/20,444,18.0,11.000000,7/20/20,4,1.0,0.285714,43.529877,1.130646
3,1007,Bibb,AL,-0.058477,22668,8/9/20,438,12.0,10.571429,8/8/20,5,1.0,0.428571,46.635912,1.890645
4,1009,Blount,AL,-0.026344,58013,7/25/20,569,17.0,21.142857,7/29/20,3,2.0,0.285714,36.445033,0.492500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2646,55137,Waushara,WI,-0.261365,24369,7/31/20,99,4.0,5.714286,8/7/20,1,1.0,0.142857,23.448996,0.586225
2647,55139,Winnebago,WI,-0.139262,170414,6/10/20,457,26.0,24.714286,5/31/20,7,1.0,0.857143,14.502497,0.502977
2648,55141,Wood,WI,-0.139262,73126,8/16/20,379,7.0,12.571429,5/23/20,1,1.0,0.142857,17.191462,0.195358
2658,56019,Johnson,WY,-0.061875,8476,3/31/20,7,2.0,1.000000,4/13/20,1,1.0,0.142857,11.798018,1.685431


In [244]:
corr_df = pd.DataFrame()

for state in state_list:
    
    df_state = df[(df['State']==state)]
    
    if len(df_state) >= 4:
    
        # covariance covariance[0,1],
        data1 = df_state['MMR']
        data2 = df_state[field_name]
        covariance = cov(data1, data2)

        # Pearsons correlation
        corr, _ = pearsonr(data1, data2)

        # R-squared
        correlation_matrix = np.corrcoef(data1, data2)
        correlation_xy = correlation_matrix[0,1]
        r_squared = correlation_xy**2

        # outputs
        corr_df = corr_df.append([[state,str(len(df_state)),str(len(mmr_df[mmr_df['State'] == state])),corr,'%.5f' % r_squared]])

corr_df.columns=['state','tested_counties', 'total_counties','pearsons','r_squared']
corr_df.sort_values(by='pearsons', ascending=True).reset_index(drop=True)

,state,tested_counties,total_counties,pearsons,r_squared
0,FL,67,67,-0.347237,0.12057
1,LA,64,64,-0.282444,0.07977
2,NC,93,100,-0.231034,0.05338
3,ID,24,44,-0.226600,0.05135
4,TN,87,95,-0.213808,0.04571
5,SC,46,46,-0.200893,0.04036
6,MD,24,24,-0.194430,0.03780
7,NJ,21,21,-0.176019,0.03098
8,IN,85,92,-0.114251,0.01305
9,KS,43,105,-0.094930,0.00901


In [245]:
# plot it

df = df4[df4[field_name]!=0]
print(len(df))

fig1 = px.scatter(df, x="MMR", y=field_name, trendline="ols")
fig1.update_traces(textposition='top center')
fig1.update_layout(
        title= field_name + ' vs. MMR',
        xaxis=dict(
            title='MMR',
            gridcolor='white',
            gridwidth=2,
        ),
        yaxis=dict(
            title=field_name,
            gridcolor='white',
            gridwidth=2,
        ),
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)', width=600, height=400
    )
fig1.show()

2088
